In [17]:
# 데이터 구성:Series, DataFrame
import pandas as pd
# 행렬 연산
import numpy as np
# 데이터 시각화
import matplotlib
import matplotlib.pyplot as plt

# Scale 변환:평균,표준편차 기준
from sklearn.preprocessing import StandardScaler
# linkage: 계층형 군집분석, dendrogram: 계층구조 나무, cluster: linkage로부터 얻은 값으로 군집 생성

import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler

# 주성분 분석
from sklearn.decomposition import PCA
# 회귀분석
import statsmodels.api as sm

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV

In [3]:
plt.rcParams['font.family'] = 'NanumGothic'
matplotlib.rcParams['axes.unicode_minus'] = False

In [4]:
df_raw = pd.read_csv('./SCALE_preprossed.csv')
df_raw.head()

,scale,spec_country,steel_kind,pt_thick,pt_width,pt_length,hsb,fur_no,fur_input_row,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,rolling_method,rolling_temp,descaling_count,work_group,bf_temp
0,양품,미국,T,32,3700,15100,적용,1호기,1열,1144,116,1133,59,259,TMCP(온도제어),934.0,8,1조,199.0
1,양품,미국,T,32,3700,15100,적용,1호기,2열,1144,122,1135,53,238,TMCP(온도제어),937.0,8,1조,198.0
2,양품,영국,T,33,3600,19200,적용,2호기,1열,1129,116,1121,55,258,TMCP(온도제어),889.0,8,1조,232.0
3,양품,영국,T,33,3600,19200,적용,2호기,2열,1152,125,1127,68,266,TMCP(온도제어),885.0,8,1조,242.0
4,양품,프랑스,T,38,3100,13300,적용,3호기,1열,1140,134,1128,48,246,TMCP(온도제어),873.0,8,1조,255.0


In [5]:
# 목표변수/설명변수 분리
df_raw_x = df_raw.drop("scale", axis = 1, inplace = False)
df_raw_y = df_raw["scale"] 

display("설명변수:", df_raw_x.head())
display("목표변수:",df_raw_y.head())

'설명변수:'

,spec_country,steel_kind,pt_thick,pt_width,pt_length,hsb,fur_no,fur_input_row,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,rolling_method,rolling_temp,descaling_count,work_group,bf_temp
0,미국,T,32,3700,15100,적용,1호기,1열,1144,116,1133,59,259,TMCP(온도제어),934.0,8,1조,199.0
1,미국,T,32,3700,15100,적용,1호기,2열,1144,122,1135,53,238,TMCP(온도제어),937.0,8,1조,198.0
2,영국,T,33,3600,19200,적용,2호기,1열,1129,116,1121,55,258,TMCP(온도제어),889.0,8,1조,232.0
3,영국,T,33,3600,19200,적용,2호기,2열,1152,125,1127,68,266,TMCP(온도제어),885.0,8,1조,242.0
4,프랑스,T,38,3100,13300,적용,3호기,1열,1140,134,1128,48,246,TMCP(온도제어),873.0,8,1조,255.0


'목표변수:'

0    양품
1    양품
2    양품
3    양품
4    양품
Name: scale, dtype: object

## 범주형 변수 제거

In [6]:
categorical_columns = df_raw_x.select_dtypes(include=['object']).columns
df_raw_x = df_raw_x.drop(columns=categorical_columns)
display(df_raw_x)

,pt_thick,pt_width,pt_length,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,rolling_temp,descaling_count,bf_temp
0,32,3700,15100,1144,116,1133,59,259,934.0,8,199.0
1,32,3700,15100,1144,122,1135,53,238,937.0,8,198.0
2,33,3600,19200,1129,116,1121,55,258,889.0,8,232.0
3,33,3600,19200,1152,125,1127,68,266,885.0,8,242.0
4,38,3100,13300,1140,134,1128,48,246,873.0,8,255.0
...,...,...,...,...,...,...,...,...,...,...,...
995,19,3400,41500,1142,55,1151,86,203,948.0,10,203.0
996,19,3400,41500,1142,55,1151,86,209,948.0,10,203.0
997,17,3400,43700,1169,65,1163,77,227,948.0,10,215.0
998,17,3400,43700,1169,65,1163,77,204,948.0,10,215.0


In [7]:
df_raw_y = df_raw_y.apply(lambda x: 0 if x == '양품' else 1)
df_raw_y

0      0
1      0
2      0
3      0
4      0
      ..
995    0
996    0
997    0
998    0
999    0
Name: scale, Length: 1000, dtype: int64

In [8]:
v_feature_names = df_raw_x.columns
scaler = StandardScaler()
df_x_scaled = scaler.fit_transform(df_raw_x)
df_x_scaled = pd.DataFrame(df_x_scaled, columns=v_feature_names)

In [9]:
df_x_scaled

,pt_thick,pt_width,pt_length,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,rolling_temp,descaling_count,bf_temp
0,0.287834,1.757877,-1.559693,-0.623753,1.140313,-1.034166,-0.617721,0.534659,-0.074240,-0.347396,-0.205910
1,0.287834,1.757877,-1.559693,-0.623753,1.368163,-0.918797,-0.909098,-0.015429,-0.031561,-0.347396,-0.222112
2,0.342996,1.555380,-1.264844,-1.330154,1.140313,-1.726379,-0.811972,0.508464,-0.714421,-0.347396,0.328734
3,0.342996,1.555380,-1.264844,-0.247005,1.482088,-1.380272,-0.180654,0.718022,-0.771325,-0.347396,0.490747
4,0.618805,0.542895,-1.689139,-0.812126,1.823863,-1.322588,-1.151913,0.194129,-0.942040,-0.347396,0.701365
...,...,...,...,...,...,...,...,...,...,...,...
995,-0.429269,1.150386,0.338846,-0.717939,-1.176161,0.004153,0.693479,-0.932242,0.124927,0.899987,-0.141105
996,-0.429269,1.150386,0.338846,-0.717939,-1.176161,0.004153,0.693479,-0.775074,0.124927,0.899987,-0.141105
997,-0.539592,1.150386,0.497058,0.553583,-0.796412,0.696366,0.256412,-0.303570,0.124927,0.899987,0.053311
998,-0.539592,1.150386,0.497058,0.553583,-0.796412,0.696366,0.256412,-0.906047,0.124927,0.899987,0.053311


In [10]:
pca = PCA(n_components=6)
nd_pca = pca.fit_transform(df_x_scaled)
# 생성된 주성분 점수 확인
print("PCA Data :\n {}".format(nd_pca[:5]))
# 생성된 주성분 점수 Row, Column 개수 확인
print("Shape :\n {}".format(nd_pca.shape))

PCA Data :
 [[ 1.34210444  0.14142801  0.3273567  -1.54939533 -2.00056098 -0.54637421]
 [ 1.07539019  0.05568835  0.09529771 -1.82426709 -2.12206535 -0.25868301]
 [ 2.02682218  0.60924727  0.21495774 -1.15687869 -2.28122467  0.37311015]
 [ 1.90719887  0.91786674  0.68630113 -1.48663964 -1.33978829 -0.2718841 ]
 [ 2.14165554  0.30111048  0.15907902 -1.90891967 -1.29225282  1.08485709]]
Shape :
 (1000, 6)


In [12]:
df_pca = pd.DataFrame(nd_pca, columns = ["Prin1","Prin2","Prin3","Prin4","Prin5","Prin6"])
# 주성분 분석용 데이터와 주성분 점수 결합
df_scaled_pca = df_x_scaled.join(df_pca)
df_scaled_pca.head()

,pt_thick,pt_width,pt_length,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,rolling_temp,descaling_count,bf_temp,Prin1,Prin2,Prin3,Prin4,Prin5,Prin6
0,0.287834,1.757877,-1.559693,-0.623753,1.140313,-1.034166,-0.617721,0.534659,-0.074240,-0.347396,-0.205910,1.342104,0.141428,0.327357,-1.549395,-2.000561,-0.546374
1,0.287834,1.757877,-1.559693,-0.623753,1.368163,-0.918797,-0.909098,-0.015429,-0.031561,-0.347396,-0.222112,1.075390,0.055688,0.095298,-1.824267,-2.122065,-0.258683
2,0.342996,1.555380,-1.264844,-1.330154,1.140313,-1.726379,-0.811972,0.508464,-0.714421,-0.347396,0.328734,2.026822,0.609247,0.214958,-1.156879,-2.281225,0.373110
3,0.342996,1.555380,-1.264844,-0.247005,1.482088,-1.380272,-0.180654,0.718022,-0.771325,-0.347396,0.490747,1.907199,0.917867,0.686301,-1.486640,-1.339788,-0.271884
4,0.618805,0.542895,-1.689139,-0.812126,1.823863,-1.322588,-1.151913,0.194129,-0.942040,-0.347396,0.701365,2.141656,0.301110,0.159079,-1.908920,-1.292253,1.084857


In [13]:
df_pca_stat = pd.DataFrame()
df_pca_stat["PrinNo"] = [I for I in range(1,7)]
df_pca_stat["EigenValues"] = pca.explained_variance_ 
df_pca_stat["EigenValueRatio"] = pca.explained_variance_ratio_ 
df_pca_stat["CumEigenValueRatio"] = np.cumsum(pca.explained_variance_ratio_)
df_pca_stat.round(3)

,PrinNo,EigenValues,EigenValueRatio,CumEigenValueRatio
0,1,4.983,0.453,0.453
1,2,1.860,0.169,0.621
2,3,1.504,0.137,0.758
3,4,0.840,0.076,0.834
4,5,0.707,0.064,0.899
5,6,0.658,0.060,0.958


적정 주성분 선택
상관계수 기준 1보다 큰 3개
공분산 기준 0.8보다 큰 건 4개

In [23]:
display(df_x_scaled.head())
display(df_raw_y.head())

,pt_thick,pt_width,pt_length,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,rolling_temp,descaling_count,bf_temp
0,0.287834,1.757877,-1.559693,-0.623753,1.140313,-1.034166,-0.617721,0.534659,-0.074240,-0.347396,-0.205910
1,0.287834,1.757877,-1.559693,-0.623753,1.368163,-0.918797,-0.909098,-0.015429,-0.031561,-0.347396,-0.222112
2,0.342996,1.555380,-1.264844,-1.330154,1.140313,-1.726379,-0.811972,0.508464,-0.714421,-0.347396,0.328734
3,0.342996,1.555380,-1.264844,-0.247005,1.482088,-1.380272,-0.180654,0.718022,-0.771325,-0.347396,0.490747
4,0.618805,0.542895,-1.689139,-0.812126,1.823863,-1.322588,-1.151913,0.194129,-0.942040,-0.347396,0.701365


0    0
1    0
2    0
3    0
4    0
Name: scale, dtype: int64

In [24]:
df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(df_x_scaled, df_raw_y
                                    , test_size = 0.3, random_state = 133)

In [34]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, StratifiedKFold

pipe = Pipeline([
    ('pca', PCA()),
    ('clf', KNeighborsClassifier()),
])

parameters = {
    'pca__n_components' : [3, 4],
    'clf__n_neighbors' : [5, 7, 9, 11, 13, 15]
}

kf = StratifiedKFold(n_splits=5, shuffle= True, random_state= 133)
grid = GridSearchCV(pipe, parameters, cv = kf, n_jobs= -1, verbose=1)
grid.fit(df_train_x, df_train_y)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=133, shuffle=True),
             estimator=Pipeline(steps=[('pca', PCA()),
                                       ('clf', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid={'clf__n_neighbors': [5, 7, 9, 11, 13, 15],
                         'pca__n_components': [3, 4]},
             verbose=1)

In [35]:
print('Best scroe : %0.3f' %grid.best_score_)
print('Best parameters set:')
best_parameters = grid.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print('\t%s: %r'%(param_name, best_parameters[param_name]))

Best scroe : 0.837
Best parameters set:
	clf__n_neighbors: 9
	pca__n_components: 4


In [39]:
from sklearn.metrics import accuracy_score
accuracy_score(df_test_y, grid.best_estimator_.predict(df_test_x))

0.8666666666666667

# 1은 불량 0은 양품

In [40]:
def results(y_pred, y_test):
    from sklearn.metrics import classification_report, confusion_matrix
    print(classification_report(y_test, y_pred))
    
results(grid.predict(df_test_x), df_test_y)

              precision    recall  f1-score   support

           0       0.88      0.95      0.91       216
           1       0.83      0.65      0.73        84

    accuracy                           0.87       300
   macro avg       0.85      0.80      0.82       300
weighted avg       0.86      0.87      0.86       300

